In [1]:
from flask import *
import pandas as pd
app = Flask(__name__)
import json
import urllib3
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from forex_python.converter import CurrencyRates

In [2]:
c = CurrencyRates()
c.get_rate('USD', 'INR')

63.61

In [3]:
# Get Todays date
i = datetime.now()
now = i.strftime('%Y-%m-%d ')

data = pd.read_excel('/Users/smahimkar/Desktop/invest/investment/invest_tables/data.xlsx')


In [4]:
data.Type.unique()

array(['Cryptocurrency', 'Deposit', 'Acorns', 'Fundrise', 'LendingClub'],
      dtype=object)

## Cryptocurrency

In [5]:
cryptocurrency_temp = data.loc[data.Type == 'Cryptocurrency']
## Get todays Value of cryptocurrencies
http = urllib3.PoolManager()
r = http.request('GET', 'https://api.coinmarketcap.com/v1/ticker/')
all_data = json.loads(r.data.decode('utf-8'))
all_data_df = pd.DataFrame(all_data)
cryptocurrency_popular = cryptocurrency_temp.merge(all_data_df[['id','price_usd']],left_on='Name',right_on='id',how='inner')
cryptocurrency_popular.Units = cryptocurrency_popular.Units.apply(float)
cryptocurrency_popular.price_usd = cryptocurrency_popular.price_usd.apply(float)
cryptocurrency_popular.Todays_Value = cryptocurrency_popular.price_usd * cryptocurrency_popular.Units
cryptocurrency_popular.drop(['id','price_usd'],axis=1,inplace=True)
cryptocurrency_popular.Maturity_Date = now

cryptocurrency_ico = cryptocurrency_temp[~cryptocurrency_temp.Name.isin(cryptocurrency_popular.Name.unique())]
cryptocurrency_ico.Maturity_Date = 'TBD'
cryptocurrency_final = pd.concat([cryptocurrency_popular,cryptocurrency_ico])
cryptocurrency_final

/Users/smahimkar/Desktop/invest/investment/flask_env/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/smahimkar/Desktop/invest/investment/flask_env/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,Name,Units,Currency,Money_Invested,Todays_Value,Date_of_Investment,Maturity_Date,Link,Type
0,ethereum,0.496452,USD,358.0,554.790075,2017-12-30,2018-01-27,myEtherWallet / Meta-mask,Cryptocurrency
1,bitcoin,0.006700,USD,2.0,77.308620,2015-05-01,2018-01-27,https://www.coinbase.com/,Cryptocurrency
2,bitcoin-cash,0.003100,USD,10.0,5.100988,2017-12-30,2018-01-27,https://www.coinbase.com/,Cryptocurrency
3,litecoin,0.649183,USD,180.0,118.322718,2017-12-30,2018-01-27,https://freewallet.org/,Cryptocurrency
4,dogecoin,8104.533848,USD,74.0,60.249510,2017-12-30,2018-01-27,https://freewallet.org/,Cryptocurrency
5,ripple,132.867000,USD,200.0,163.282914,2018-01-20,2018-01-27,https://www.binance.com/,Cryptocurrency
6,sonocoin,86.000000,USD,129.0,129.000000,2017-12-30,TBD,https://sono.money/,Cryptocurrency
7,crypterium,101.000000,USD,160.0,160.000000,2017-12-30,TBD,https://crypterium.io/,Cryptocurrency
8,rentberry,476.000000,USD,106.0,106.000000,2017-12-30,TBD,https://cryptonomos.com/wallets/eth,Cryptocurrency
9,locicoin,110.720000,USD,89.6,89.600000,2017-12-30,TBD,https://sale.locipro.com/,Cryptocurrency


In [6]:
deposit_temp = data.loc[data.Type == 'Deposit']
deposit_temp.Currency = 'USD'
deposit_temp.Money_Invested = deposit_temp.Money_Invested / c.get_rate('USD', 'INR')
deposit_temp.Todays_Value = deposit_temp.Todays_Value /  c.get_rate('USD', 'INR')
deposit_temp

/Users/smahimkar/Desktop/invest/investment/flask_env/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,Name,Units,Currency,Money_Invested,Todays_Value,Date_of_Investment,Maturity_Date,Link,Type
10,icici_rd,NaN,USD,4716.239585,4895.236598,2017-07-03,2018-07-03 00:00:00,https://www.icicibank.com,Deposit


In [7]:
def get_monthly_investment_factor(date_of_investment):
    monthly_investment_factor = pd.to_datetime(now) - pd.to_datetime(date_of_investment) 
    mif = monthly_investment_factor.days // 30.0
    return mif


In [41]:
acorns_data = data.loc[data.Type == 'Acorns']
mif = get_monthly_investment_factor(acorns_data.Date_of_Investment.values[0])
acorns_data.Money_Invested = acorns_data.Money_Invested + (mif * 100.0)
acorns_data.Todays_Value = acorns_data.Money_Invested + (acorns_data.Money_Invested * 3) / 100.0
acorns_data.Maturity_Date = now
acorns_data

/Users/smahimkar/Desktop/invest/investment/flask_env/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,Name,Units,Currency,Money_Invested,Todays_Value,Date_of_Investment,Maturity_Date,Link,Type
11,acorns,NaN,USD,696.97,717.8791,2018-01-01,2018-01-27,https://www.acorns.com/,Acorns


In [10]:
fundrise_data = data.loc[data.Type == 'Fundrise']
mif = get_monthly_investment_factor(fundrise_data.Date_of_Investment.values[0])
fundrise_data.Money_Invested = fundrise_data.Money_Invested + (mif * 100.0)
fundrise_data.Todays_Value = fundrise_data.Money_Invested + (fundrise_data.Money_Invested * 0.7) / 100.0
fundrise_data

/Users/smahimkar/Desktop/invest/investment/flask_env/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,Name,Units,Currency,Money_Invested,Todays_Value,Date_of_Investment,Maturity_Date,Link,Type
12,fundrise,NaN,USD,500.0,503.5,2018-01-01,Quarterly,https://fundrise.com/,Fundrise


In [9]:
lendingClub = data.loc[data.Type == 'LendingClub']
mif = get_monthly_investment_factor(lendingClub.Date_of_Investment.values[0])
lendingClub.Money_Invested = lendingClub.Money_Invested + (mif * 100.0)
lendingClub.Todays_Value = lendingClub.Money_Invested + ((lendingClub.Money_Invested * 5.61) /1200.0) * 0 
lendingClub

/Users/smahimkar/Desktop/invest/investment/flask_env/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,Name,Units,Currency,Money_Invested,Todays_Value,Date_of_Investment,Maturity_Date,Link,Type
13,LendingClub,NaN,USD,1000.0,1000.0,2018-01-01,Monthly,https://www.lendingclub.com/,LendingClub


In [45]:
new_data = pd.concat([cryptocurrency_final,deposit_temp,acorns,fundrise,lendingClub])
# new_data.set_index(['Name'], inplace=True)
# new_data.index.name=None
# cryptocurrency = new_data.loc[new_data.Type == 'Cryptocurrency']
# deposit = new_data.loc[new_data.Type == 'Deposit']

In [46]:
new_data

,Name,Units,Currency,Money_Invested,Todays_Value,Date_of_Investment,Maturity_Date,Link,Type
0,ethereum,0.496452,USD,358.000000,549.319173,2017-12-30 00:00:00,2018-01-27,myEtherWallet / Meta-mask,Cryptocurrency
1,bitcoin,0.006700,USD,2.000000,77.224200,2015-05-01 00:00:00,2018-01-27,https://www.coinbase.com/,Cryptocurrency
2,bitcoin-cash,0.003100,USD,10.000000,5.102259,2017-12-30 00:00:00,2018-01-27,https://www.coinbase.com/,Cryptocurrency
3,litecoin,0.649183,USD,180.000000,117.811161,2017-12-30 00:00:00,2018-01-27,https://freewallet.org/,Cryptocurrency
4,dogecoin,8104.533848,USD,74.000000,60.101116,2017-12-30 00:00:00,2018-01-27,https://freewallet.org/,Cryptocurrency
5,ripple,132.867000,USD,200.000000,163.293543,2018-01-20 00:00:00,2018-01-27,https://www.binance.com/,Cryptocurrency
6,sonocoin,86.000000,USD,129.000000,129.000000,2017-12-30 00:00:00,TBD,https://sono.money/,Cryptocurrency
7,crypterium,101.000000,USD,160.000000,160.000000,2017-12-30 00:00:00,TBD,https://crypterium.io/,Cryptocurrency
8,rentberry,476.000000,USD,106.000000,106.000000,2017-12-30 00:00:00,TBD,https://cryptonomos.com/wallets/eth,Cryptocurrency
9,locicoin,110.720000,USD,89.600000,89.600000,2017-12-30 00:00:00,TBD,https://sale.locipro.com/,Cryptocurrency


In [47]:
total = new_data.groupby('Currency').agg({'Money_Invested':'sum','Todays_Value':'sum'}).reset_index()
total

,Currency,Money_Invested,Todays_Value
0,USD,8171.809585,8522.567151
